### 📘 Example Prompts (Try one of these book titles):
- The Da Vinci Code  
- Wild Animus  
- The Lovely Bones: A Novel  
- The Red Tent  
- A Time to Kill


In [ ]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix
import zipfile
import urllib.request
import os

# downloading and extracting dataset
data_url = "https://github.com/shaswat373/Book-Recommendation-AI/raw/main/data.zip"
data_zip_path = "data.zip"

if not os.path.exists(data_zip_path):
    print("Downloading dataset...")
    urllib.request.urlretrieve(data_url, data_zip_path)
    print("Dataset downloaded.")

with zipfile.ZipFile(data_zip_path, 'r') as zip_ref:
    zip_ref.extractall()

books_df = pd.read_csv("Books.csv", low_memory=False)
ratings_df = pd.read_csv("Ratings.csv", low_memory=False)

book_counts = ratings_df['ISBN'].value_counts()
valid_books = book_counts[book_counts >= 30].index
ratings_df = ratings_df[ratings_df['ISBN'].isin(valid_books)]

book_user_matrix = ratings_df.pivot_table(
    index='ISBN', columns='User-ID', values='Book-Rating'
).fillna(0)

book_user_sparse = csr_matrix(book_user_matrix.values)
similarity_matrix = cosine_similarity(book_user_sparse)
isbn_list = book_user_matrix.index.tolist()
isbn_to_title = books_df.set_index('ISBN')['Book-Title'].to_dict()

def recommend_books(book_title, top_n=5):
    matching_books = books_df[books_df['Book-Title'].str.lower() == book_title.lower()]

    if matching_books.empty:
        return f"Sorry, no book found with the title '{book_title}'. Please check the spelling."

    target_isbn = matching_books.iloc[0]['ISBN']

    if target_isbn not in isbn_list:
        return f"We found the book '{book_title}', but it does not have enough rating data to generate recommendations."

    idx = isbn_list.index(target_isbn)
    similarity_scores = list(enumerate(similarity_matrix[idx]))
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)[1:top_n+1]

    recommended_books = []
    for i, score in similarity_scores:
        isbn = isbn_list[i]
        title = isbn_to_title.get(isbn, "Unknown Title")
        recommended_books.append((title, round(float(score), 3)))

    return recommended_books

book_title = input("Enter a book title you like: ")

results = recommend_books(book_title)

if isinstance(results, str):
    print(results)
else:
    print(f"\nIf you liked '{book_title}', you might also enjoy:\n")
    for i, (title, score) in enumerate(results, 1):
        print(f"{i}. {title}")
